In [ ]:
#Constants
DATA_PATH = "/Users/Hashem/Link TSP/Project1/data/raw/survey_results_public.csv"
EXPORT_PATH = "/Users/Hashem/Link TSP/Project1/data/processed/1_preprocessed_df.pkl"

REPLACE_DICT ={
    'YearsCodePro' : {'Less than 1 year' : 0, 'More than 50 years' : 51},
    'YearsCode' : {'Less than 1 year' : 0, 'More than 50 years' : 51},
    'Age1stCode' : {'Older than 85' : 86, 'Younger than 5 years' : 4},
}

Libraries

In [2]:
#Load packages
import pandas as pd
import numpy as np
import logging
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_rows = 10000

Function

In [14]:
def split_answers(data_series, delimiter=','):
    """
    Split multiple answer is a signal string
    to a list of single each represnting a single answer

    parameters:
    data_series (pd.Series): String series with answers
    delimiter (string): Another decimal integer
                        Defaults to ";"
    
    Returns: (pd.Series): If column contains
    """
    #sub function
    def is_splittable(pd_series, delimiter):
        """ Check if results multiple should be splitted - Returns boolean """
        return pd_series.str.contains(delimiter)

    def split_answer(pd_series, delimiter):
        """ Function to split signle answer"""
        return pd_series.str.split(delimiter)

    #-------------------------------------------
    
    #check if multiple answer exist - if name: return original
    splittable_values = is_splittable(data_series, delimiter)
    if not splittable_values.any():
        return data_series
    
    #Else, split each value to a list
    modified_series = split_answer(data_series, delimiter)

    #replace NAs with empty lists
    mask_null = modified_series.isnull()
    modified_series.loc[mask_null] = modified_series.loc[mask_null].apply(lambda x: [])
    
    return modified_series

Processing

Preprocess Data

In [15]:
raw_df= pd.read_csv(DATA_PATH)
df = raw_df.copy()
#df.head(5)
#df.columns
df['DevType'].head(20)

0     Developer, desktop or enterprise applications;...
1               Developer, full-stack;Developer, mobile
2                                                   NaN
3                                                   NaN
4                                                   NaN
5       Designer;Developer, front-end;Developer, mobile
6     Developer, back-end;Developer, front-end;Devel...
7     Developer, back-end;Developer, desktop or ente...
8                                 Developer, full-stack
9     Database administrator;Developer, full-stack;D...
10    Developer, back-end;Developer, desktop or ente...
11                        Designer;Developer, front-end
12                         Designer;Developer, back-end
13    Developer, desktop or enterprise applications;...
14                                                  NaN
15    Database administrator;Developer, back-end;Dev...
16                                Developer, full-stack
17                                  Developer, b

1. Replace Values and parse

In [16]:
for col, replacement in REPLACE_DICT.items():
    df[col]= df[col].replace(replacement).astype(np.float32)

2. Split multiple answers

In [18]:
object_cols = df.select_dtypes(include='object').columns.tolist()

for col in object_cols:
    df[col] = split_answers(df[col])

Export Data

In [21]:
df.to_pickle(EXPORT_PATH)

In [20]:
df['DevType'].head(20)

0     [Developer,  desktop or enterprise application...
1           [Developer,  full-stack;Developer,  mobile]
2                                                    []
3                                                    []
4                                                    []
5     [Designer;Developer,  front-end;Developer,  mo...
6     [Developer,  back-end;Developer,  front-end;De...
7     [Developer,  back-end;Developer,  desktop or e...
8                              [Developer,  full-stack]
9     [Database administrator;Developer,  full-stack...
10    [Developer,  back-end;Developer,  desktop or e...
11                     [Designer;Developer,  front-end]
12                      [Designer;Developer,  back-end]
13    [Developer,  desktop or enterprise application...
14                                                   []
15    [Database administrator;Developer,  back-end;D...
16                             [Developer,  full-stack]
17                               [Developer,  ba